# Comparing different methods of approximating nearest neighbors

## Euclidean Space

Python implementation of different searches for exact nearest neighbors using random numbers between 0 and 1000 for a dataset of size 1000.
These functions will select a random point from the dataset and find its nearest neighbor. 
They'll then print the selected point, its nearest neighbor, and the Euclidean distance between them.

### Method 1: Exhaustive Search

In [1]:
import numpy as np
import time

def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

def exhaustive_nearest_neighbors(dataset):
    num_points = len(dataset)
    nearest_neighbors = []

    for i in range(num_points):
        min_distance = float('inf')
        nearest_neighbor_idx = None

        for j in range(num_points):
            if i != j:
                distance = euclidean_distance(dataset[i], dataset[j])
                if distance < min_distance:
                    min_distance = distance
                    nearest_neighbor_idx = j
        
        nearest_neighbors.append(nearest_neighbor_idx)
    
    return nearest_neighbors

def main():
    np.random.seed(42)  # Set random seed for reproducibility
    dataset_size = 1000
    dataset = np.random.randint(0, 1001, size=(dataset_size, 2))  # Generating random points

    selected_point_idx = np.random.randint(0, dataset_size)  # Select a random point
    selected_point = dataset[selected_point_idx]
    
    start_time = time.time()
    nearest_neighbors = exhaustive_nearest_neighbors(dataset)
    end_time = time.time()

    nearest_neighbor_idx = nearest_neighbors[selected_point_idx]
    nearest_neighbor = dataset[nearest_neighbor_idx]
    distance_to_nearest_neighbor = euclidean_distance(selected_point, nearest_neighbor)

    print("Selected Point:", selected_point)
    print("Nearest Neighbor:", nearest_neighbor)
    print("Distance to Nearest Neighbor:", distance_to_nearest_neighbor)
    print("Time taken:", end_time - start_time, "seconds")

if __name__ == "__main__":
    main()

Selected Point: [710 400]
Nearest Neighbor: [709 415]
Distance to Nearest Neighbor: 15.033296378372908
Time taken: 10.629792928695679 seconds


### Method 2: [scikit-learn's NearestNeighbors](https://scikit-learn.org/stable/modules/neighbors.html)

In [ ]:
pip install scikit-learn

In [6]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import time

def main():
    np.random.seed(42)  # Set random seed for reproducibility
    dataset_size = 1000
    embedding_size = 2

    # Generate random data points for the dataset
    dataset = np.random.randint(0, 1001, size=(dataset_size, embedding_size)).astype(np.float32)

    # Build the NearestNeighbors index
    nn = NearestNeighbors(n_neighbors=2, algorithm='brute', metric='euclidean')
    nn.fit(dataset)

    selected_point_idx = np.random.randint(0, dataset_size)  # Select a random point
    selected_point = dataset[selected_point_idx]

    start_time = time.time()
    
    # Find the nearest neighbor
    distances, indices = nn.kneighbors([selected_point])
    nearest_neighbor_idx = indices[0, 1]
    nearest_neighbor = dataset[nearest_neighbor_idx]
    distance_to_nearest_neighbor = distances[0, 1]
    
    end_time = time.time()

    print("Selected Point:", selected_point)
    print("Nearest Neighbor:", nearest_neighbor)
    print("Distance to Nearest Neighbor:", distance_to_nearest_neighbor)
    print("Time taken:", end_time - start_time, "seconds")

if __name__ == "__main__":
    main()

Selected Point: [710. 400.]
Nearest Neighbor: [709. 415.]
Distance to Nearest Neighbor: 15.033297
Time taken: 0.13064813613891602 seconds


### Method 3: [SciPy's KDTree](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html) 

In [7]:
pip install scipy

In [8]:
import numpy as np
from scipy.spatial import KDTree
import time

def main():
    np.random.seed(42)  # Set random seed for reproducibility
    dataset_size = 1000
    embedding_size = 2

    # Generate random data points for the dataset
    dataset = np.random.randint(0, 1001, size=(dataset_size, embedding_size)).astype(np.float32)

    # Build the KDTree
    kdtree = KDTree(dataset)

    selected_point_idx = np.random.randint(0, dataset_size)  # Select a random point
    selected_point = dataset[selected_point_idx]

    start_time = time.time()
    
    # Find the nearest neighbor
    distance, nearest_neighbor_idx = kdtree.query(selected_point, k=2)
    nearest_neighbor = dataset[nearest_neighbor_idx[1]]
    
    end_time = time.time()

    print("Selected Point:", selected_point)
    print("Nearest Neighbor:", nearest_neighbor)
    print("Distance to Nearest Neighbor:", distance[1])
    print("Time taken:", end_time - start_time, "seconds")

if __name__ == "__main__":
    main()

Selected Point: [710. 400.]
Nearest Neighbor: [709. 415.]
Distance to Nearest Neighbor: 15.033296378372908
Time taken: 0.02311539649963379 seconds


### Method 4: [pynndescent's NNDescent](https://pynndescent.readthedocs.io/en/latest/)

In [9]:
pip install pynndescent

Note: you may need to restart the kernel to use updated packages.Collecting pynndescent

  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55638 sha256=c532f6fa40d260c9d7d9acef9ea9a3f776699786bc4d9b0f5cb0f625fbeed6b5
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\12\f9\4d\ec5ad1c823c710fcc4473669fdcffc8891f4bc398c841af22e
Successfully built pynndescent


In [10]:
import numpy as np
from pynndescent import NNDescent
import time

def main():
    np.random.seed(42)  # Set random seed for reproducibility
    dataset_size = 1000
    embedding_size = 2

    # Generate random data points for the dataset
    dataset = np.random.randint(0, 1001, size=(dataset_size, embedding_size)).astype(np.float32)

    # Build the NNDescent index
    nnd_index = NNDescent(dataset)

    selected_point_idx = np.random.randint(0, dataset_size)  # Select a random point
    selected_point = dataset[selected_point_idx]

    start_time = time.time()
    
    # Find the nearest neighbor
    nearest_neighbor_idx, _ = nnd_index.query([selected_point], k=2)
    nearest_neighbor = dataset[nearest_neighbor_idx[0, 1]]
    
    end_time = time.time()

    print("Selected Point:", selected_point)
    print("Nearest Neighbor:", nearest_neighbor)
    print("Approximate Distance to Nearest Neighbor:", np.linalg.norm(selected_point - nearest_neighbor))
    print("Time taken:", end_time - start_time, "seconds")

if __name__ == "__main__":
    main()

c:\Users\ASUS\anaconda3\envs\Pandas\lib\site-packages\numba\np\ufunc\parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


Selected Point: [242.  85.]
Nearest Neighbor: [230.  85.]
Approximate Distance to Nearest Neighbor: 12.0
Time taken: 19.835765600204468 seconds


### Method 5: [FAISS by Meta](https://github.com/facebookresearch/faiss)

So far, this is the fastest method.

In [2]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import faiss
import time

def main():
    np.random.seed(42)  # Set random seed for reproducibility
    dataset_size = 1000
    embedding_size = 2

    # Generate random data points for the dataset
    dataset = np.random.randint(0, 1001, size=(dataset_size, embedding_size)).astype(np.float32)

    # Build the Faiss index
    index = faiss.IndexFlatL2(embedding_size)
    index.add(dataset)

    selected_point_idx = np.random.randint(0, dataset_size)  # Select a random point
    selected_point = dataset[selected_point_idx]

    start_time = time.time()
    
    # Find the nearest neighbor
    distance, nearest_neighbor_idx = index.search(np.array([selected_point]), k=2)
    nearest_neighbor = dataset[nearest_neighbor_idx[0, 1]]
    
    end_time = time.time()

    print("Selected Point:", selected_point)
    print("Nearest Neighbor:", nearest_neighbor)
    print("Approximate Distance to Nearest Neighbor:", distance[0, 1])
    print("Time taken:", end_time - start_time, "seconds")

if __name__ == "__main__":
    main()

Selected Point: [710. 400.]
Nearest Neighbor: [709. 415.]
Approximate Distance to Nearest Neighbor: 226.0
Time taken: 0.0009987354278564453 seconds


#### Besides, you can also try [Annoy (developed by Spotify)](https://github.com/spotify/annoy), [NMSLIB](https://github.com/nmslib/nmslib), [NGT (developed by YahooJapan)](https://github.com/yahoojapan/NGT). For some reasons, my C+ compiler has not been able to execute these codes. 

## Hyperbolic Space

### Method 1: [Geomloss Library](https://www.kernel-operations.io/geomloss/)

In [7]:
pip install geomloss

  Created wheel for geomloss: filename=geomloss-0.2.6-py3-none-any.whl size=32259 sha256=24f30d799693e4c1fe86253af18d11d300d97cf3f167161e1295cd6d0a124a99
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\6f\e1\ba\7ecd1fe2056dc36c59f58b7c9f2ca2075abd585caa5cd83ce6
Successfully built geomloss
Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import torch
from geomloss import SamplesLoss
import time

def main():
    np.random.seed(42)  # Set random seed for reproducibility
    dataset_size = 1000
    embedding_size = 2

    # Generate random data points in hyperbolic space
    hyperbolic_points = np.random.normal(size=(dataset_size, embedding_size))
    hyperbolic_points = torch.tensor(hyperbolic_points, dtype=torch.float32)

    # Select a random point
    selected_point_idx = np.random.randint(0, dataset_size)
    selected_point = hyperbolic_points[selected_point_idx]

    # Generate another random point in hyperbolic space
    random_point_idx = np.random.randint(0, dataset_size)
    random_point = hyperbolic_points[random_point_idx]

    start_time = time.time()

    # Perform nearest neighbor search
    loss = SamplesLoss(loss="sinkhorn", p=2)
    distances = loss(selected_point.unsqueeze(0), random_point.unsqueeze(0))

    end_time = time.time()

    print("Selected Point:", selected_point)
    print("Random Point:", random_point)
    print("Approximate Distance to Random Point:", distances.item())
    print("Time taken:", end_time - start_time, "seconds")

if __name__ == "__main__":
    main()

Selected Point: tensor([ 1.5796, -0.5229])
Random Point: tensor([-0.9717, -1.3796])
Approximate Distance to Random Point: 3.621401786804199
Time taken: 0.07400345802307129 seconds


### Other possible approaches: [HNSWLIB](https://github.com/nmslib/hnswlib), [Poincare-Embeddings](https://github.com/facebookresearch/poincare-embeddings) 